<a href="https://colab.research.google.com/github/chadloh/GoogleDecimeterChallenge/blob/main/CarolineKeough/IMU_based_on_2021_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict Next Point with the IMU Data
Author: **Alvin.ai**<br>
IMU is Inertial Measurement Unit, whicn involves accelerometer, gyroscope asn so on. In Microsoft Research's Indoor Location & Navigation Competition, many kagglers used the IMU Data. In recent days, I attempted to build a model to utilize the given sensors' data and I found that it works for me as below:<br>
1. Reject Outlier + KF Smooth + Phone Mean: LB-**5.653**
2. **IMU Prediction** + Reject Outlier + KF Smooth + Phone Mean: LB-**5.476**

What I do is to simple, **use historical points with sensors' dataset to predict where the next point is.**<br>
Talk is cheap, show you the code. Hope it is helpful to you.<br>
By the way, **I am looking forward to find a teammates, if you're interested, pls see the end of this kernal.**<br>


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 16.9 MB/s 


In [4]:
import math
import numpy as np
from cv2 import Rodrigues
from math import sin, cos, atan2, sqrt
import pandas as pd
from pathlib import Path
import pyproj
from pyproj import Proj, transform
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from tqdm import tqdm
import warnings
import itertools
warnings.filterwarnings("ignore", category=Warning)

In [5]:
#Select files to train. Try to have different locations, routes, phones
#LAX, MTV, SJC, SVL, SFO

data_dir = Path('/content/drive/MyDrive/Google_decimeter/smartphone-decimeter-2022')

train1 = Path('2021-12-28-US-MTV-1/SamsungGalaxyS20Ultra')
train2 = Path('2021-12-15-US-MTV-1/GooglePixel5')
train3 = Path('2021-12-09-US-LAX-2/GooglePixel6Pro')
train4 = Path('2021-12-08-US-LAX-5/XiaomiMi8')
train5 = Path('2021-08-24-US-SVL-1/GooglePixel4')
train6 = Path('2021-08-04-US-SJC-1/GooglePixel5')
train7 = Path('2021-07-27-US-MTV-1/XiaomiMi8')
train8 = Path('2021-04-26-US-SVL-2/SamsungGalaxyS20Ultra')
train9 = Path('2021-04-29-US-MTV-2/SamsungGalaxyS20Ultra')
train10 = Path('2021-01-04-US-SFO-2/GooglePixel5') #this one won't have orientation deg

In [6]:
pd.read_csv(data_dir / 'train' / train3 / 'supplemental' / 'gnss_log.txt', sep = ' ')

ParserError: ignored

In [ ]:
#select test files. as in from train folder. but to test the model
test1 = Path('2021-12-07-US-LAX-1/GooglePixel5')
test2 = Path('2021-07-14-US-MTV-1/SamsungGalaxyS20Ultra')

test_sets = [test1, test2]

In [ ]:
#this was an attempt to do list append instead of df concat

# list_of_training_sets = [train1, train2]#, 'train3', 'train4', 'train5',\
#                         # 'train6', 'train7', 'train8', 'train9', 'train10']

# # def get_bls(list_of_training_sets):
# # bl_trn_df = pd.DataFrame(columns = ['collectionName', 'phoneName','utctimeMillis', 'Xbl', 'Ybl', 'Zbl'])
# collectionName =[]
# phoneName = []
# utcTimeMillis = []
# Xbl = []
# Ybl = []
# Zbl = []

# for train in list_of_training_sets:
#   device_gnss = pd.read_csv(data_dir / train / 'device_gnss.csv')\
#   [['utcTimeMillis', 'WlsPositionXEcefMeters','WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']]
#   device_gnss = device_gnss.drop_duplicates().reset_index().drop(columns = 'index')
#   device_gnss = device_gnss.rename(columns = {'WlsPositionXEcefMeters':'Xbl',\
#                 'WlsPositionYEcefMeters':'Ybl', 'WlsPositionZEcefMeters': 'Ybl'}) 
#   utcTimeMillis.append(device_gnss['utcTimeMillis'])
#   collectionName.append([str(train).split('/')[0]]*len(device_gnss))
#   phoneName.append([str(train).split('/')[1]]*len(device_gnss))
#   Xbl.append(device_gnss[])
#   # bl_trn_df = pd.concat([bl_trn_df, bl], axis = 0)
#   #do I need to set_index('collectionName')???
# bl_trn_df = pd.DataFrame(list(zip(collectionName, phoneName, utcTimeMillis, Xbl, Ybl, Zbl)))


In [ ]:
#internet said not to concat dfs but i'm gonna try
training_sets = [train1, train2, train3, train4, train5,\
                         train6, train7, train8, train9, train10]

def get_bls(list_of_training_sets):
  bl_trn_df = pd.DataFrame(columns = ['collectionName', 'phoneName','utcTimeMillis', 'Xbl', 'Ybl', 'Zbl'])

  for train in list_of_training_sets:
    #just updated line below hopefully still works
    device_gnss = pd.read_csv(data_dir / 'train' / train / 'device_gnss.csv')\
    [['utcTimeMillis', 'WlsPositionXEcefMeters','WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']]
    device_gnss = device_gnss.drop_duplicates().reset_index().drop(columns = 'index')
    device_gnss = device_gnss.rename(columns = {'WlsPositionXEcefMeters':'Xbl',\
                  'WlsPositionYEcefMeters':'Ybl', 'WlsPositionZEcefMeters': 'Zbl'}) 
    collectionName = pd.Series([str(train).split('/')[0]]*len(device_gnss), name = 'collectionName')
    phoneName = pd.Series([str(train).split('/')[1]]*len(device_gnss), name = 'phoneName')
    bl = pd.concat([collectionName, phoneName, device_gnss], axis = 1)
    bl_trn_df = pd.concat([bl_trn_df, bl], axis = 0, ignore_index = True)
  return bl_trn_df


In [ ]:
bl_trn_df = get_bls(training_sets)

,collectionName,phoneName,utcTimeMillis,Xbl,Ybl,Zbl
0,2021-12-28-US-MTV-1,SamsungGalaxyS20Ultra,1640722632000,-2.693857e+06,-4.297547e+06,3.854144e+06
1,2021-12-28-US-MTV-1,SamsungGalaxyS20Ultra,1640722633000,-2.693856e+06,-4.297544e+06,3.854144e+06
2,2021-12-28-US-MTV-1,SamsungGalaxyS20Ultra,1640722634000,-2.693855e+06,-4.297539e+06,3.854143e+06
3,2021-12-28-US-MTV-1,SamsungGalaxyS20Ultra,1640722635000,-2.693860e+06,-4.297547e+06,3.854146e+06
4,2021-12-28-US-MTV-1,SamsungGalaxyS20Ultra,1640722636000,-2.693858e+06,-4.297548e+06,3.854148e+06
...,...,...,...,...,...,...
18153,2021-01-04-US-SFO-2,GooglePixel5,1609801853433,-2.704197e+06,-4.288807e+06,3.856680e+06
18154,2021-01-04-US-SFO-2,GooglePixel5,1609801854433,-2.704197e+06,-4.288810e+06,3.856682e+06
18155,2021-01-04-US-SFO-2,GooglePixel5,1609801855433,-2.704196e+06,-4.288806e+06,3.856680e+06
18156,2021-01-04-US-SFO-2,GooglePixel5,1609801856433,-2.704196e+06,-4.288804e+06,3.856678e+06


In [ ]:
bl_tst_df = get_bls(test_sets)

In [ ]:
# data_dir = Path("../input/google-smartphone-decimeter-challenge")
# bl_trn_fname = 'baseline_locations_train.csv'
# bl_tst_fname = 'baseline_locations_test.csv'
# sample_fname = 'sample_submission.csv'

In [ ]:
# bl_trn_df = pd.read_csv(data_dir / bl_trn_fname)
# bl_tst_df = pd.read_csv(data_dir / bl_tst_fname)


#FLAG do i need to do sample df
# sample_df = pd.read_csv(data_dir / sample_fname)

In [ ]:
# print('Baseline Train shape:', bl_trn_df.shape)
# print('Baseline Test shape:', bl_tst_df.shape)
# print('Test shape:', sample_df.shape)

## 1. Euler Angles to Rotation Vector
Euler Angles <-> Rotation Matrix <-> Rotation Vector<br>
More info.:[About IMU: OrientationDeg to Rotation Vector](https://www.kaggle.com/c/google-smartphone-decimeter-challenge/discussion/247834)

In [ ]:
# pitch:y
# yaw:z
# roll:x
def an2v(y_delta, z_delta, x_delta):
    '''
    Euler Angles ->Rotation Matrix -> Rotation Vector

    Input：
        1. y_delta          (float): the angle with rotateing around y-axis.
        2. z_delta         (float): the angle with rotateing around z-axis. 
        3. x_delta         (float): the angle with rotateing around x-axis. 
    Output：
        rx/ry/rz             (float): the rotation vector with rotateing 
    
    Code Ref.: https://www.zacobria.com/universal-robots-knowledge-base-tech-support-forum-hints-tips/python-code-example-of-converting-rpyeuler-angles-to-rotation-vectorangle-axis-for-universal-robots/
    (Note：In Code Ref: pitch=y,yaw=z,roll=x. But Google is pitch=x,yaw=z,roll=y)
    '''
    # yaw: z
    Rz_Matrix = np.matrix([
    [math.cos(z_delta), -math.sin(z_delta), 0],
    [math.sin(z_delta), math.cos(z_delta), 0],
    [0, 0, 1]
    ])
    
    # pitch: y
    Ry_Matrix = np.matrix([
    [math.cos(y_delta), 0, math.sin(y_delta)],
    [0, 1, 0],
    [-math.sin(y_delta), 0, math.cos(y_delta)]
    ])
    
    # roll: x
    Rx_Matrix = np.matrix([
    [1, 0, 0],
    [0, math.cos(x_delta), -math.sin(x_delta)],
    [0, math.sin(x_delta), math.cos(x_delta)]
    ])

    R = Rz_Matrix * Ry_Matrix * Rx_Matrix

    theta = math.acos(((R[0, 0] + R[1, 1] + R[2, 2]) - 1) / 2)
    multi = 1 / (2 * math.sin(theta))

    rx = multi * (R[2, 1] - R[1, 2]) * theta
    ry = multi * (R[0, 2] - R[2, 0]) * theta
    rz = multi * (R[1, 0] - R[0, 1]) * theta

    return rx, ry, rz

In [ ]:
def v2a(rotation_v):
    '''
    Rotation Vector -> Rotation Matrix -> Euler Angles

    Input：
        rx/ry/rz             (float): the rotation vector with rotateing around x/y/z-axis.
    Output：
        1. y_delta          (float): the angle with rotateing around y-axis.
        2. z_delta         (float): the angle with rotateing around z-axis. 
        3. x_delta         (float): the angle with rotateing around x-axis.  
    '''
    # Rotation Vector -> Rotation Matrix
    R = Rodrigues(rotation_v)[0]

    sq = sqrt(R[2,1] ** 2 +  R[2,2] ** 2)

    if  not (sq < 1e-6) :
        x_delta = atan2(R[2,1] , R[2,2])
        y_delta = atan2(-R[2,0], sq)
        z_delta = atan2(R[1,0], R[0,0])
    else :
        x_delta = atan2(-R[1,2], R[1,1])
        y_delta = atan2(-R[2,0], sq)
        z_delta = 0

    return y_delta, z_delta, x_delta

## 2. Prepare IMU Dataset
This part is to prepare the dataset for the model. I divided this part into the following steps:<br>
(1) **Load GNSS Log**<br>
(2) **Merge sub-dataset** (Status/UncalAccel/UncalGyro/UncalMag/OrientationDeg)<br>
(3) **UTC to GpsEpoch**<br>
(4) **OrientationDeg to Rotation Vecto**r<br>
(5) **Calibrate Sensors' data**<br>
(6) **LatDeg&lngDeg to x/y/z**<br>
(7) **Orgainze Data** (eg. t1 t2 t3 t4 t5 -> t6)<br>
(8) **Clean Data** (unrelated-aixs features and uncalibrated features)<br>
(9) **Add Statistic Features**

In [ ]:
# # gnss_log_to_dataframes(str(data_dir / 'train' / tgt_cns[0] / pns[0] / 'supplemental' / 'gnss_log.txt'))
# # data_dir, 'train', tgt_cns[0], pns[0], bl_trn_df

# path = str(data_dir / 'train' / tgt_cns[0] / pns[0] / 'supplemental' / 'gnss_log.txt')

# gnss_section_names = {'Raw', 'UncalAccel', 'UncalGyro', 'UncalMag', 'Fix', 'Status', 'OrientationDeg'}

# with open(path) as f_open:
#   datalines = f_open.readlines()
# datas = {k: [] for k in gnss_section_names}
# gnss_map = {k: [] for k in gnss_section_names}
# for dataline in datalines:
#   if dataline.rstrip():
#     is_header = dataline.startswith('#')
#     dataline = dataline.strip('#').strip().split(',')
#     # skip over notes, version numbers, etc
#     if is_header and dataline[0] in gnss_section_names:
#         gnss_map[dataline[0]] = dataline[1:]
#     elif not is_header:
#         datas[dataline[0]].append(dataline[1:])


In [ ]:
def gnss_log_to_dataframes(path):
    '''Load GNSS Log'''
    print('Loading ' + path, flush = True)
    gnss_section_names = {'Raw', 'UncalAccel', 'UncalGyro', 'UncalMag', 'Fix', 'Status', 'OrientationDeg'}
    with open(path) as f_open:
        datalines = f_open.readlines()

    datas = {k: [] for k in gnss_section_names}
    gnss_map = {k: [] for k in gnss_section_names}
    for dataline in datalines:
      if dataline.rstrip():
        is_header = dataline.startswith('#')
        dataline = dataline.strip('#').strip().split(',')
        # skip over notes, version numbers, etc
        if is_header and dataline[0] in gnss_section_names:
            gnss_map[dataline[0]] = dataline[1:]
        elif not is_header:
            datas[dataline[0]].append(dataline[1:])

    results = dict()
    for k, v in datas.items():
        results[k] = pd.DataFrame(v, columns=gnss_map[k])
    # pandas doesn't properly infer types from these lists by default
    for k, df in results.items():
        for col in df.columns:
            if col == 'CodeType':
                continue
            results[k][col] = pd.to_numeric(results[k][col])

    return results

In [ ]:
def UTC2GpsEpoch(df):
    '''UTC to GpsEpoch
    
    utcTimeMillis         : UTC epoch (1970/1/1)
    millisSinceGpsEpoch   : GPS epoch(1980/1/6 midnight 12:00 UTC)
    
    Ref: https://www.kaggle.com/c/google-smartphone-decimeter-challenge/discussion/239187
    '''
    dt_offset = pd.to_datetime('1980-01-06 00:00:00') 
    dt_offset_in_ms = int(dt_offset.value / 1e6)
    df['millisSinceGpsEpoch'] = df['utcTimeMillis'] - dt_offset_in_ms + 18000
    return df

In [218]:
gnss_df = gnss_log_to_dataframes(str(data_dir / 'train' / tgt_cns[0] / pns[0] / 'supplemental' / 'gnss_log.txt'))

print('sub-dataset shape：')
print('Raw:', gnss_df['Raw'].shape)
print('Status:', gnss_df['Status'].shape)
print('UncalAccel:', gnss_df['UncalAccel'].shape)
print('UncalGyro:', gnss_df['UncalGyro'].shape)
print('UncalMag:', gnss_df['UncalMag'].shape)
print('OrientationDeg:', gnss_df['OrientationDeg'].shape)
print('Fix:', gnss_df['Fix'].shape)

Loading /content/drive/MyDrive/Google_decimeter/smartphone-decimeter-2022/train/2021-12-28-US-MTV-1/SamsungGalaxyS20Ultra/supplemental/gnss_log.txt
sub-dataset shape：
Raw: (62648, 36)
Status: (0, 13)
UncalAccel: (99351, 8)
UncalGyro: (80754, 8)
UncalMag: (161508, 8)
OrientationDeg: (0, 5)
Fix: (0, 12)


In [223]:
dataset_name = 'train'
cname = tgt_cns[0]
pname = pns[0]

In [224]:
imu_df = pd.merge_asof(gnss_df['UncalAccel'].sort_values('utcTimeMillis'),
                        gnss_df['UncalGyro'].drop('elapsedRealtimeNanos', axis=1).sort_values('utcTimeMillis'),
                        on = 'utcTimeMillis',
                        direction='nearest')
# (accel + gyro) + mag
imu_df = pd.merge_asof(imu_df.sort_values('utcTimeMillis'),
                        gnss_df['UncalMag'].drop('elapsedRealtimeNanos', axis=1).sort_values('utcTimeMillis'),
                        on = 'utcTimeMillis',
                        direction='nearest')
# ((accel + gyro) + mag) + OrientationDeg
imu_df = pd.merge_asof(imu_df.sort_values('utcTimeMillis'),
                        gnss_df['OrientationDeg'].drop('elapsedRealtimeNanos', axis=1).sort_values('utcTimeMillis'),
                        on = 'utcTimeMillis',
                        direction='nearest')

imu_df = imu_df.rename(columns = {'utcTimeMillis': 'UnixTimeMillis'})

if dataset_name == 'train':
    # read GT dataset
    gt_path = data_dir / dataset_name / cname / pname / 'ground_truth.csv'
    #might need to add cols of course and phone before unixtime millis
    gt_df = pd.read_csv(gt_path, usecols = ['UnixTimeMillis', 'LatitudeDegrees', 'LongitudeDegrees'])


    # print GT time
    # tmp_datetime = pd.to_datetime(gt_df['millisSinceGpsEpoch'] + dt_offset_in_ms, unit='ms')
    # print(f"gt_df time scope: {tmp_datetime.min()} - {tmp_datetime.max()}")

    # merge GT dataset
    imu_df = pd.merge_asof(gt_df.sort_values('UnixTimeMillis'),
                            imu_df.drop(['elapsedRealtimeNanos'], axis=1).sort_values('UnixTimeMillis'),
                            on = 'UnixTimeMillis',
                            direction='nearest')
elif dataset_name == 'test':
    # merge smaple_df
    imu_df = pd.merge_asof(sample_df.sort_values('UnixTimeMillis'),
                        imu_df.drop(['elapsedRealtimeNanos'], axis=1).sort_values('UnixTimeMillis'),
                        on = 'UnixTimeMillis',
                        direction='nearest')


    

In [231]:
pd.read_csv()

,LatitudeDegrees,LongitudeDegrees,UnixTimeMillis,UncalAccelXMps2,UncalAccelYMps2,UncalAccelZMps2,BiasXMps2,BiasYMps2,BiasZMps2,UncalGyroXRadPerSec,...,DriftZRadPerSec,UncalMagXMicroT,UncalMagYMicroT,UncalMagZMicroT,BiasXMicroT,BiasYMicroT,BiasZMicroT,yawDeg,rollDeg,pitchDeg
0,37.41603,-122.08094,1640722632000,-0.153229,9.665691,-2.063802,0.0,0.0,0.0,0.000153,...,0.0,81.659996,-57.780000,-32.100000,51.0,-14.88,-56.76,NaN,NaN,NaN
1,37.41603,-122.08094,1640722633000,-0.134075,9.620202,-2.056619,0.0,0.0,0.0,-0.004734,...,0.0,82.080000,-57.420000,-32.219997,51.0,-14.88,-56.76,NaN,NaN,NaN
2,37.41603,-122.08094,1640722634000,-0.155623,9.648932,-2.123657,0.0,0.0,0.0,-0.001069,...,0.0,81.780000,-57.719997,-32.040000,51.0,-14.88,-56.76,NaN,NaN,NaN
3,37.41603,-122.08094,1640722635000,-0.148441,9.601048,-2.068590,0.0,0.0,0.0,-0.005956,...,0.0,81.420000,-57.660000,-31.920000,51.0,-14.88,-56.76,NaN,NaN,NaN
4,37.41603,-122.08094,1640722636000,-0.093374,9.567530,-2.111686,0.0,0.0,0.0,-0.002291,...,0.0,81.840000,-57.600000,-32.460000,51.0,-14.88,-56.76,NaN,NaN,NaN


In [ ]:
def prepare_imu_data(data_dir, dataset_name, cname, pname, bl_df):
    '''Prepare IMU Dataset (For Train: IMU+GT+BL; For Test: IMU+BL)
    Input：
        1. data_dir: data_dir
        2. dataset_name: dataset name（'train'/'test'）
        3. cname: CollectionName
        4. pname: phoneName
        5. bl_df: baseline's dataframe
    Output：df_all
    '''
    # load GNSS log
    gnss_df = gnss_log_to_dataframes(str(data_dir / dataset_name / cname / pname / 'supplemental' / 'gnss_log.txt'))
    print('sub-dataset shape：')
    print('Raw:', gnss_df['Raw'].shape)
    print('Status:', gnss_df['Status'].shape)
    print('UncalAccel:', gnss_df['UncalAccel'].shape)
    print('UncalGyro:', gnss_df['UncalGyro'].shape)
    print('UncalMag:', gnss_df['UncalMag'].shape)
    print('OrientationDeg:', gnss_df['OrientationDeg'].shape)
    print('Fix:', gnss_df['Fix'].shape)

    # merge sub-datasets
    # accel + gyro
    imu_df = pd.merge_asof(gnss_df['UncalAccel'].sort_values('utcTimeMillis'),
                           gnss_df['UncalGyro'].drop('elapsedRealtimeNanos', axis=1).sort_values('utcTimeMillis'),
                           on = 'utcTimeMillis',
                           direction='nearest')
    # (accel + gyro) + mag
    imu_df = pd.merge_asof(imu_df.sort_values('utcTimeMillis'),
                           gnss_df['UncalMag'].drop('elapsedRealtimeNanos', axis=1).sort_values('utcTimeMillis'),
                           on = 'utcTimeMillis',
                           direction='nearest')
    # ((accel + gyro) + mag) + OrientationDeg
    imu_df = pd.merge_asof(imu_df.sort_values('utcTimeMillis'),
                           gnss_df['OrientationDeg'].drop('elapsedRealtimeNanos', axis=1).sort_values('utcTimeMillis'),
                           on = 'utcTimeMillis',
                           direction='nearest')
    
    imu_df = imu_df.rename(columns = {'utcTimeMillis': 'UnixTimeMillis'})
    
    # UTC->GpsEpoch
    # imu_df = UTC2GpsEpoch(imu_df)
    # I think I can just leave it in UTC cuz everything is in that.

    # print IMU time
    # dt_offset = pd.to_datetime('1980-01-06 00:00:00')
    # dt_offset_in_ms = int(dt_offset.value / 1e6)
    # tmp_datetime = pd.to_datetime(imu_df['millisSinceGpsEpoch'] + dt_offset_in_ms, unit='ms')
    # print(f"imu_df time scope: {tmp_datetime.min()} - {tmp_datetime.max()}")


    if dataset_name == 'train':
        # read GT dataset
        gt_path = data_dir / dataset_name / cname / pname / 'ground_truth.csv'
        #might need to add cols of course and phone before unixtime millis
        gt_df = pd.read_csv(gt_path, usecols = ['UnixTimeMillis', 'LatitudeDegrees', 'LongitudeDegrees'])


        # print GT time
        # tmp_datetime = pd.to_datetime(gt_df['millisSinceGpsEpoch'] + dt_offset_in_ms, unit='ms')
        # print(f"gt_df time scope: {tmp_datetime.min()} - {tmp_datetime.max()}")

        # merge GT dataset
        imu_df = pd.merge_asof(gt_df.sort_values('UnixTimeMillis'),
                               imu_df.drop(['elapsedRealtimeNanos'], axis=1).sort_values('UnixTimeMillis'),
                               on = 'UnixTimeMillis',
                               direction='nearest')
    elif dataset_name == 'test':
        # merge smaple_df
        imu_df = pd.merge_asof(sample_df.sort_values('UnixTimeMillis'),
                           imu_df.drop(['elapsedRealtimeNanos'], axis=1).sort_values('UnixTimeMillis'),
                           on = 'UnixTimeMillis',
                           direction='nearest')

    # OrientationDeg -> Rotation Vector
    rxs = []
    rys = []
    rzs = []
    for i in range(len(imu_df)):
        y_delta = imu_df['rollDeg'].iloc[i]
        z_delta = imu_df['yawDeg'].iloc[i]
        x_delta = imu_df['pitchDeg'].iloc[i]
        rx, ry, rz = an2v(y_delta, z_delta, x_delta)
        rxs.append(rx)
        rys.append(ry)
        rzs.append(rz)

    imu_df['ahrsX'] = rxs
    imu_df['ahrsY'] = rys
    imu_df['ahrsZ'] = rzs

    # calibrate sensors' reading
    for axis in ['X', 'Y', 'Z']:
        imu_df['Accel{}Mps2'.format(axis)] = imu_df['UncalAccel{}Mps2'.format(axis)] - imu_df['Bias{}Mps2'.format(axis)]
        imu_df['Gyro{}RadPerSec'.format(axis)] = imu_df['UncalGyro{}RadPerSec'.format(axis)] - imu_df['Drift{}RadPerSec'.format(axis)]
        imu_df['Mag{}MicroT'.format(axis)] = imu_df['UncalMag{}MicroT'.format(axis)] - imu_df['Bias{}MicroT'.format(axis)]

        # clearn bias features
        imu_df.drop(['Bias{}Mps2'.format(axis), 'Drift{}RadPerSec'.format(axis), 'Bias{}MicroT'.format(axis)], axis = 1, inplace = True) 
#FLAG... I don't have latdeg for bl i have xyz. see if later on its turned to xyz.
    bl_df.rename(columns = {'utcTimeMillis': 'UnixTimeMillis'})
    if dataset_name == 'train':
        # merge Baseline dataset：imu_df + bl_df = (GT + IMU) + Baseline
        df_all = pd.merge(imu_df.rename(columns={'LatitudeDegree':'latDeg_gt', 'LongitudeDegree':'lngDeg_gt'}),
                      bl_df,
                      on = ['collectionName', 'phoneName', 'UnixTimeMillis'])
    elif dataset_name == 'test':
        df_all = pd.merge(imu_df,
              bl_df[(bl_df['collectionName']==cname) & (bl_df['phoneName']==pname)],
              on = ['UnixTimeMillis'])
        # df_all.drop(['phone'], axis=1, inplace=True)
        
    return df_all

In [ ]:
df_all()

NameError: ignored

In [ ]:
def WGS84_to_ECEF(lat, lon, alt):
    # convert to radians
    rad_lat = lat * (np.pi / 180.0)
    rad_lon = lon * (np.pi / 180.0)
    a    = 6378137.0
    # f is the flattening factor
    finv = 298.257223563
    f = 1 / finv   
    # e is the eccentricity
    e2 = 1 - (1 - f) * (1 - f)    
    # N is the radius of curvature in the prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
    x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
    y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
    z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
    return x, y, z

transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)
def ECEF_to_WGS84(x,y,z):
    lon, lat, alt = transformer.transform(x,y,z,radians=False)
    return lon, lat, alt

In [ ]:
def get_xyz(df_all, dataset_name):
    # baseline: lat/lngDeg -> x/y/z
    #FLAG might want to just delete this first line here
    df_all['Xbl'], df_all['Ybl'], df_all['Zbl'] = zip(*df_all.apply(lambda x: WGS84_to_ECEF(x.latDeg_bl, x.lngDeg_bl, x.heightAboveWgs84EllipsoidM), axis=1))
    
    if dataset_name == 'train':
        # gt: lat/lngDeg -> x/y/z
        df_all['Xgt'], df_all['Ygt'], df_all['Zgt'] = zip(*df_all.apply(lambda x: WGS84_to_ECEF(x.latDeg_gt, x.lngDeg_gt, x.heightAboveWgs84EllipsoidM), axis=1))
        # copy lat/lngDeg
        lat_lng_df = df_all[['latDeg_gt','lngDeg_gt', 'latDeg_bl', 'lngDeg_bl']]
        #FLAG will need to modify here. Do I need to put baseline into lat long?
        df_all.drop(['latDeg_gt','lngDeg_gt', 'latDeg_bl', 'lngDeg_bl'], axis = 1, inplace = True)
    elif dataset_name == 'test':
        # copy lat/lngDeg
        lat_lng_df = df_all[['latDeg_bl', 'lngDeg_bl']]
        #FLAG here too
        df_all.drop(['latDeg_bl', 'lngDeg_bl', 'latDeg','lngDeg',], axis = 1, inplace = True)     
        
    return lat_lng_df, df_all

In [ ]:
def prepare_df_train(df_all_train, window_size):
    '''prepare training dataset with all aixses'''
    tgt_df = df_all_train.copy()
    total_len = len(tgt_df) 
    moving_times = total_len - window_size 
    
    tgt_df.rename(columns = {'yawDeg':'yawZDeg', 'rollDeg':'rollYDeg', 'pitchDeg':'pitchXDeg'}, inplace = True)

    feature_cols = [f for f in list(tgt_df) if f not in ['Xgt', 'Ygt', 'Zgt']]

    # Historical Feature names
    hist_feats = []
    for time_flag in range(1, window_size + 1):
        for fn in feature_cols:
            hist_feats.append(fn + '_' + str(time_flag))

    # Window Sliding
    # t1 t2 t3 t4 t5 -> t6
    # t2 t3 t4 t5 t6 -> t7

    # Add historical data 
    df_train = pd.DataFrame()
    features = []
    xs = []
    ys = []
    zs = []

    for start_idx in range(moving_times):
        feature_list = list()
        x_list = list()
        y_list = list()
        z_list = list()

        for window_idx in range(window_size):
            feature_list.extend(tgt_df[feature_cols].iloc[start_idx + window_idx,:].to_list())
        x_list.append(tgt_df['Xgt'].iloc[start_idx + window_size])
        y_list.append(tgt_df['Ygt'].iloc[start_idx + window_size])
        z_list.append(tgt_df['Zgt'].iloc[start_idx + window_size])

        features.append(feature_list)
        xs.extend(x_list)
        ys.extend(y_list)
        zs.extend(z_list)

    df_train = pd.DataFrame(features, columns = hist_feats)
    df_train['Xgt'] = xs
    df_train['Ygt'] = ys
    df_train['Zgt'] = zs
    
    # clean single-value feature: collectionName_[1-5]\phoneName_[1-5]
    tmp_feats = []
    for fn in list(df_train):
        if (fn.startswith('collectionName_') == False) and (fn.startswith('phoneName_') == False):
            tmp_feats.append(fn)
    df_train = df_train[tmp_feats]

    # clean time feature
    tmp_drop_feats = []
    for f in list(df_train):
        if (f.startswith('millisSinceGpsEpoch') == True) or (f.startswith('timeSinceFirstFixSeconds') == True) or (f.startswith('utcTimeMillis') == True):
            tmp_drop_feats.append(f)
    df_train.drop(tmp_drop_feats, axis = 1, inplace = True)
    
    return df_train

In [ ]:
def prepare_df_test(df_all_test, window_size):
    '''prepare testing dataset with all aixses'''
    tgt_df = df_all_test.copy()
    total_len = len(tgt_df) 
    moving_times = total_len - window_size 
    
    tgt_df.rename(columns = {'yawDeg':'yawZDeg', 'rollDeg':'rollYDeg', 'pitchDeg':'pitchXDeg'}, inplace = True)

    feature_cols = [f for f in list(tgt_df) if f not in ['Xgt', 'Ygt', 'Zgt']] 
    
    hist_feats = []
    for time_flag in range(1, window_size + 1):
        for fn in feature_cols:
            hist_feats.append(fn + '_' + str(time_flag))

    # t1 t2 t3 t4 t5 -> t6
    # t2 t3 t4 t5 t6 -> t7
    df_test = pd.DataFrame()
    features = []

    for start_idx in range(moving_times):
        feature_list = list()

        for window_idx in range(window_size):
            feature_list.extend(tgt_df[feature_cols].iloc[start_idx + window_idx,:].to_list())
        features.append(feature_list)

    df_test = pd.DataFrame(features, columns = hist_feats)

    tmp_feats = []
    for fn in list(df_test):
        if (fn.startswith('collectionName_') == False) and (fn.startswith('phoneName_') == False):
            tmp_feats.append(fn)
    df_test = df_test[tmp_feats]

    tmp_drop_feats = []
    for f in list(df_test):
        if (f.startswith('millisSinceGpsEpoch') == True) or (f.startswith('timeSinceFirstFixSeconds') == True) or (f.startswith('utcTimeMillis') == True) or (f.startswith('elapsedRealtimeNanos') == True):
            tmp_drop_feats.append(f)
    df_test.drop(tmp_drop_feats, axis = 1, inplace = True)
    
    return df_test

In [ ]:
def remove_other_axis_feats(df_all, tgt_axis):
    '''unrelated-aixs features and uncalibrated features'''
    # Clean unrelated-aixs features
    all_imu_feats = ['UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2',
                     'UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec',
                     'UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT',
                     'ahrsX', 'ahrsY', 'ahrsZ',
                     'AccelXMps2', 'AccelYMps2', 'AccelZMps2',
                     'GyroXRadPerSec', 'GyroZRadPerSec', 'GyroYRadPerSec',
                     'MagXMicroT', 'MagYMicroT', 'MagZMicroT',
                     'yawZDeg', 'rollYDeg', 'pitchXDeg',
                     'Xbl', 'Ybl', 'Zbl']
    tgt_imu_feats = []
    for axis in ['X', 'Y', 'Z']:
        if axis != tgt_axis:
            for f in all_imu_feats:
                if f.find(axis) >= 0:
                    tgt_imu_feats.append(f)
            
    tmp_drop_feats = []
    for f in list(df_all):
        if f.split('_')[0] in tgt_imu_feats:
            tmp_drop_feats.append(f)

    tgt_df = df_all.drop(tmp_drop_feats, axis = 1)
    
    # Clean uncalibrated features
    uncal_feats = [f for f in list(tgt_df) if f.startswith('Uncal') == True]
    tgt_df = tgt_df.drop(uncal_feats, axis = 1)
    
    return tgt_df

In [ ]:
def add_stat_feats(data, tgt_axis):
    for f in ['yawZDeg', 'rollYDeg', 'pitchXDeg']:
        if f.find(tgt_axis) >= 0:
            ori_feat = f
            break
            
    cont_feats = ['heightAboveWgs84EllipsoidM', 'ahrs{}'.format(tgt_axis),
           'Accel{}Mps2'.format(tgt_axis), 'Gyro{}RadPerSec'.format(tgt_axis), 'Mag{}MicroT'.format(tgt_axis),
            '{}bl'.format(tgt_axis)] + [ori_feat]
    
    for f in cont_feats:
        data[f + '_' + str(window_size) + '_mean'] = data[[f + f'_{i}' for i in range(1,window_size)]].mean(axis=1)
        data[f + '_' + str(window_size) + '_std'] = data[[f + f'_{i}' for i in range(1,window_size)]].std(axis=1)
        data[f + '_' + str(window_size) + '_max'] = data[[f + f'_{i}' for i in range(1,window_size)]].max(axis=1)
        data[f + '_' + str(window_size) + '_min'] = data[[f + f'_{i}' for i in range(1,window_size)]].min(axis=1)
        data[f + '_' + str(window_size) + '_median'] = data[[f + f'_{i}' for i in range(1,window_size)]].median(axis=1)
    return data

## 3. Modeling
Note: I only use the given axis features for predict the target axis location.<br>
For example, use features contains x-axis to predict the next x location.<br>
More, I used LGBM here.

In [ ]:
# LightGBM
params = {
    'metric':'mse',
    'objective':'regression',
    'seed':2021,
    'boosting_type':'gbdt',
    'early_stopping_rounds':10,
    'subsample':0.7,
    'feature_fraction':0.7,
    'bagging_fraction': 0.7,
    'reg_lambda': 10
}
window_size = 30
verbose_flag = True
folds = 5

In [ ]:
tgt_cns = [str(x).split('/')[0] for x in training_sets]
pns = [str(x).split('/')[1] for x in training_sets]

In [ ]:
# Example: I use SJC's dataset for training 
# tgt_cns = ['2021-04-22-US-SJC-1', '2021-04-28-US-SJC-1', '2021-04-29-US-SJC-2']
cn2pn_df = bl_trn_df[['collectionName', 'phoneName']].drop_duplicates()

df_trains = []
lat_lng_df_trains = []
for tgt_cn, tgt_pn in zip(tgt_cns, pns):
  print('Prepare Training Dataset：', tgt_cn + '_' + tgt_pn)  
  df_all_train = prepare_imu_data(data_dir, 'train', tgt_cn, tgt_pn, bl_trn_df)
#   lat_lng_df_train, df_all_train = get_xyz(df_all_train, 'train')
#   df_train = prepare_df_train(df_all_train,  window_size) # 所有轴的数据
#   df_trains.append(df_train)
#   lat_lng_df_trains.append(lat_lng_df_train)
#   print('_'*20)
        
# df_train = pd.concat(df_trains, axis = 0)
# lat_lng_df_train = pd.concat(lat_lng_df_trains, axis = 0)
# print('Final Dataset shape：', df_train.shape)

Prepare Training Dataset： 2021-12-28-US-MTV-1_SamsungGalaxyS20Ultra
Loading /content/drive/MyDrive/Google_decimeter/smartphone-decimeter-2022/train/2021-12-28-US-MTV-1/SamsungGalaxyS20Ultra/supplemental/gnss_log.txt
sub-dataset shape：
Raw: (62648, 36)
Status: (0, 13)
UncalAccel: (99351, 8)
UncalGyro: (80754, 8)
UncalMag: (161508, 8)
OrientationDeg: (0, 5)
Fix: (0, 12)


KeyError: ignored

In [ ]:
# Example: I choose one of SJC collection from the test dataset as my test dataset, you can choose what as you like
cname_test = '2021-04-29-US-SJC-3'
pname_test = 'SamsungS20Ultra'
df_all_test = prepare_imu_data(data_dir, 'test', cname_test, pname_test, bl_tst_df)
lat_lng_df_test, df_all_test = get_xyz(df_all_test, 'test')
df_test = prepare_df_test(df_all_test,  window_size)
print('df_test:', df_test.shape)
print('df_test.columns:', df_test.columns)

In [ ]:
def training(df_train, df_test, tgt_axis, window_size):
    '''For the given axis target to train the model. Also, it has validation and prediciton.'''
    df_train = remove_other_axis_feats(df_train, tgt_axis)
    df_train = add_stat_feats(df_train, tgt_axis)
    df_test = remove_other_axis_feats(df_test, tgt_axis)
    df_test = add_stat_feats(df_test, tgt_axis)
    
    feature_names = [f for f in list(df_train) if f not in ['Xgt', 'Ygt', 'Zgt']]
    target = '{}gt'.format(tgt_axis)

    kfold = KFold(n_splits=folds, shuffle=True, random_state=params['seed'])

    pred_valid = np.zeros((len(df_train),)) 
    pred_test = np.zeros((len(df_test),)) 
    scores = []
    for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train, df_train[target])):
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][target]
        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][target]

        model = lgb.LGBMRegressor(**params)
        lgb_model = model.fit(X_train, 
                              Y_train,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=0,
                              eval_metric=params['metric'],
                              early_stopping_rounds=params['early_stopping_rounds'])

        pred_valid[val_idx] = lgb_model.predict(X_val, num_iteration =  lgb_model.best_iteration_)
        pred_test += lgb_model.predict(df_test[feature_names], num_iteration =  lgb_model.best_iteration_)

        scores.append(lgb_model.best_score_['valid']['l2'])
    
    pred_test = pred_test /  kfold.n_splits
    
    if verbose_flag == True:
        print("Each Fold's MSE：{}, Average MSE：{:.4f}".format([np.round(v,2) for v in scores], np.mean(scores)))
        print("-"*60)
    return df_train, df_test, pred_valid, pred_test

In [ ]:
df_train_x, df_test_x, pred_valid_x, pred_test_x = training(df_train, df_test, 'X', window_size)
df_train_y, df_test_y, pred_valid_y, pred_test_y = training(df_train, df_test, 'Y', window_size)
df_train_z, df_test_z, pred_valid_z, pred_test_z = training(df_train, df_test, 'Z', window_size)

In [ ]:
val_compare_df = pd.DataFrame({'Xgt':df_train_x['Xgt'].values, 'Xpred':pred_valid_x,
                               'Ygt':df_train_y['Ygt'].values, 'Ypred':pred_valid_y,
                                'Zgt':df_train_z['Zgt'].values, 'Zpred':pred_valid_z})

In [ ]:
val_compare_df[['Zgt', 'Zpred']].plot(figsize=(16,8))

In [ ]:
val_compare_df[['Ygt', 'Ypred']].plot(figsize=(16,8))

We can see that, gt and pre is close to each other.

In [ ]:
# xyz -> lng, lat
lng_gt, lat_gt, _ = ECEF_to_WGS84(val_compare_df['Xgt'].values,val_compare_df['Ygt'].values,val_compare_df['Zgt'].values)
lng_pred, lat_pred, _ = ECEF_to_WGS84(val_compare_df['Xpred'].values,val_compare_df['Ypred'].values,val_compare_df['Zpred'].values)
lng_test_pred, lat_test_pred, _ = ECEF_to_WGS84(pred_test_x, pred_test_y, pred_test_z)

    
val_compare_df['latDeg_gt'] = lat_gt
val_compare_df['lngDeg_gt'] = lng_gt
val_compare_df['latDeg_pred'] = lat_pred
val_compare_df['lngDeg_pred'] = lng_pred
test_pred_df = pd.DataFrame({'latDeg':lat_test_pred, 'lngDeg':lng_test_pred})

In [ ]:
# From：https://www.kaggle.com/emaerthin/demonstration-of-the-kalman-filter
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(a**0.5)
    dist = 6_367_000 * c
    return dist

In [ ]:
# Baseline vs. GT
lat_lng_df_train['dist'] = calc_haversine(lat_lng_df_train.latDeg_gt, lat_lng_df_train.lngDeg_gt, 
                                lat_lng_df_train.latDeg_bl, lat_lng_df_train.lngDeg_bl)
print('dist_50:',np.percentile(lat_lng_df_train['dist'],50) )
print('dist_95:',np.percentile(lat_lng_df_train['dist'],95) )
print('avg_dist_50_95:',(np.percentile(lat_lng_df_train['dist'],50) + np.percentile(lat_lng_df_train['dist'],95))/2)
print('avg_dist:', lat_lng_df_train['dist'].mean())

In [ ]:
# IMU Prediction vs. GT
val_compare_df['dist'] = calc_haversine(val_compare_df.latDeg_gt, val_compare_df.lngDeg_gt, 
                                val_compare_df.latDeg_pred, val_compare_df.lngDeg_pred)
# IMU预测vsGT（多collection）
print('dist_50:',np.percentile(val_compare_df['dist'],50) )
print('dist_95:',np.percentile(val_compare_df['dist'],95) )
print('avg_dist_50_95:',(np.percentile(val_compare_df['dist'],50) + np.percentile(val_compare_df['dist'],95))/2)
print('avg_dist:', val_compare_df['dist'].mean())

In [ ]:
def visualize_trafic(df, center, zoom=15):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df):
    target_df = df
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [ ]:
# Visualization: Train dataset
cname = '2021-04-29-US-SJC-2'
pname = 'SamsungS20Ultra'
# IMU Prediciton
tmp0 = val_compare_df.copy()
tmp0.rename(columns={'latDeg_pred':'latDeg', 'lngDeg_pred':'lngDeg'}, inplace=True)
tmp0['phoneName'] = [cname + '_' + pname + '_imu_pred' for i in range(len(tmp0))]
# GT
tmp1 = val_compare_df.copy()
tmp1.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'}, inplace=True)
tmp1['phoneName'] = [cname + '_' + pname + '_gt' for i in range(len(tmp1))]
# Baseline
tmp2 = lat_lng_df_train.copy()
tmp2.rename(columns={'latDeg_bl':'latDeg', 'lngDeg_bl':'lngDeg'}, inplace=True)
tmp2['phoneName'] = [cname + '_' + pname + '_bl_pred' for i in range(len(tmp2))]

tmp = pd.concat([tmp0, tmp1, tmp2])
visualize_collection(tmp)

In [ ]:
# Visualization:: Test dataset
cname = '2021-04-29-US-SJC-3'
pname = 'SamsungS20Ultra'
tmp3 = test_pred_df.copy()
tmp3['phoneName'] = cname_test + '_' + pname_test + '_imu_pred' 

tmp4 = bl_tst_df.iloc[bl_tst_df[bl_tst_df['phone']==cname_test + '_' + pname_test].index[window_size:],3:5].copy()
tmp4['phoneName'] = cname_test + '_' + pname_test + '_bl_pred' 

tmp5 = pd.concat([tmp3, tmp4])
visualize_collection(tmp5)

## 4. Submission

In [ ]:
bl_tst_df.iloc[bl_tst_df[bl_tst_df['phone']==cname_test + '_' + pname_test].index[window_size:],3] = test_pred_df['latDeg'].values
bl_tst_df.iloc[bl_tst_df[bl_tst_df['phone']==cname_test + '_' + pname_test].index[window_size:],4] = test_pred_df['lngDeg'].values

# bl_tst_df.to_csv('../submit/imu_baseline_locations_test.csv', index = False)

The above code is just a template. I cannot promise it could improve a lot for you because i didn't publish my entire code (which contains some tricks). However, i hope my code could inspire you how to use GNSS log data. <br>

On the other hand, i am seeking for a nice teammate, here is my basic infomation:
1. I am fresh of Kaggle, but attend some competitions in China before.
2. I am Data Analyst from OPPO for more than one year (well, indoor game uses our phone's data).
3. I am graduated from University of Manchester, UK. I guess my English can handle the normal communication.
4. I have another teammate from china, too. (he is nice)
5. I can use 1-3 hrs/day for playing this competition but the weekend i got one day for fee.

What I hope:
1. You are Top 50 or related experience about phone locating.
2. You have time and energy to keep working on the game.
3. You are like to publish the code when we finished this game.
4. Most importance!!! you have great ideas!

If you want to join us, feel free to contact me. My email: **alvinai9603@outlook.com**<br>

Thanks.